In [ ]:
import bentoml
import numpy as np

Dans ce corrigé, nous allons principalement nous concentrer sur le code du service (à stocker dans un fichier service.py ou avec un autre nommage )

In [ ]:
# En premier lieu, nous définissons la liste des régions de notre périmètre
regions = ["Auvergne-Rhône-Alpes", "Île-de-France", "Nouvelle-Aquitaine", "Occitanie","Provence-Alpes-Côte d'Azur"]

Ensuite, nous définissons notre service

In [ ]:
@bentoml.service()
class TransactionPredictionRegion:
    

    def __init__(self):
        transaction_classifiers = { region :
            bentoml.models.get(
        "transaction_below_market_identifier_{region}:latest"
    ) for region in list_regions
        }
        transaction_regressors = { region :
            bentoml.models.get(
        "transaction_value_estimator_{region}:latest"
    ) for region in list_regions
        }

    @bentoml.api
    def predict(self, region, transactions: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        
        self.classifier = bentoml.sklearn.load_model(self.transaction_classifiers[region])
        self.regressor = bentoml.sklearn.load_model(self.transaction_regressors[region])
        predicted_classes = self.classifier.predict(transactions)
        predicted_transactions = self.regressor.predict(transactions)
        return (predicted_transactions, predicted_classes)


Deux approches (entre autres) sont possibles : 
* Dès que le service sera lancé (avec la commande bentoml serve), tous les modèles (5 de classification et 5 de régression) seront d'abord chargés. C'est ce que nous avons fait ici. 
* Nous ne chargeons que les 2 modèles pertinents, au moment où un utilisateur appelle l'API pour l'inférence

Il n'y a pas de bons choix dans l'absolu, tout dépend des contraintes de projet (en termes de vitesse de réponse souhaitée de l'API, du volume du trafic que l'API va encaisser, etc.). 

Ici, nous avons procédé avec la 1ere approche, car les modèles sont relativement légers et que nous avons peu de régions.

Une approche plus scalable serait d'avoir plusieurs instances actives de la même API, chacune servant une région 

Vu la définition du endpoint, il est attendu à ce que nous fournissions comme input une région et un ensemble d'observations sous la forme d'un numpy array. Nous pouvons, comme vu dans le cours, 
* Utiliser le terminal avec curl 
* Utiliser le Swagger UI 
* Utiliser Python, avec la librairie requests directement ou indirectement via BentoMl (comme fait ci-dessous)

In [ ]:
'''
Après avoir lancé la commande bentoml serve, nous pouvons intéragir avec
le modèle en précisant la région et les transactions à prédire.
'''

with bentoml.SyncHTTPClient("http://localhost:3000") as client:
    result = client.predict(region = "Occitanie", transactions=test_values)
